In [50]:
import pandas as pd 
import numpy as np 
import os
import torch
from torch import nn

# Deep Neural Networks Laboratory

## A first, shallow Neural Network

To check if the CUDA version of Pytorch was installed successfully, only if your machine supports CUDA, you can do:

In [51]:
torch.cuda.is_available()   

True

The input of a neural network will be composed of two tensors: the sample and the label.

Imagine we have a classification task with 3 possible labels (0,1,2) and samples of 5 features (5-dimensional samples).

We will have tensors like:

In [52]:
x = torch.rand(1,5)
y = torch.randint(3, (1,)).long()

In [53]:
x

tensor([[0.9588, 0.0216, 0.7873, 0.8269, 0.5765]])

In [54]:
y

tensor([2])

Let us define a shallow, very simple Neural Network.

The Neural Network will have: 
1. As many input neurons as sample features (5 in this case)
2. An hidden layer 
3. As many ouput neurons as possible labels (3 in this case)

![images/shallow_nn.png](images/shallow_nn.png) 

This schema entails 4 different operations:
1. Matrix multiplication with the weights of the connections from the input layer to the FC Layer
2. Activation function in the FC Layer
3. Matrix multiplication with the weights of the connections from the FC Layer to the output layer
4. Activation function in the output layer

In [55]:
from torch import nn

In [56]:
class ShallowNeuralNetwork(nn.Module):
    def __init__(self, input_size, num_hidden, output_size):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.fc1 = nn.Linear(input_size, num_hidden)
        self.output_layer = nn.Linear(num_hidden, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        f0 = self.fc1(x)
        h0 = self.relu(f0)
        o1 = self.output_layer(h0)
        y_hat = self.softmax(o1)
        return y_hat

![images/shallow_nn.png](images/shallow_nn.png) 

## Step by step computation of a forward pass

So, we start with the tensor x.

In [57]:
x

tensor([[0.9588, 0.0216, 0.7873, 0.8269, 0.5765]])

x goes through the hidden layer. What this means is that we obtain the output:

$ Wx + b $ where:
1. W is the weight matrix of the connections
2. b is the bias vector

These values depend on the chosen initialization and might be random at first.

In [58]:
hidden_layer = nn.Linear(5,10)

print( hidden_layer.weight)

print( hidden_layer.bias )

Parameter containing:
tensor([[ 0.3557,  0.1669,  0.3688, -0.0225,  0.2863],
        [ 0.1758,  0.1700, -0.3691,  0.2275,  0.3567],
        [ 0.4320, -0.1515, -0.2495,  0.4450, -0.3763],
        [-0.3540, -0.0040,  0.2255,  0.2718, -0.3134],
        [-0.0704, -0.0580,  0.0332, -0.0148, -0.4016],
        [ 0.0399,  0.1599,  0.0599, -0.2924,  0.0257],
        [-0.0979, -0.0152,  0.2841,  0.1063, -0.1893],
        [ 0.3256,  0.3841, -0.1064, -0.1068,  0.0494],
        [-0.3044,  0.1710,  0.0316, -0.1863,  0.2877],
        [ 0.2414,  0.3849,  0.1046,  0.2524, -0.0562]], requires_grad=True)
Parameter containing:
tensor([ 0.0872, -0.2679,  0.2346, -0.0049,  0.1185,  0.0639,  0.1569,  0.1561,
         0.1971,  0.1991], requires_grad=True)


In [59]:
print( hidden_layer.weight.T)

tensor([[ 0.3557,  0.1758,  0.4320, -0.3540, -0.0704,  0.0399, -0.0979,  0.3256,
         -0.3044,  0.2414],
        [ 0.1669,  0.1700, -0.1515, -0.0040, -0.0580,  0.1599, -0.0152,  0.3841,
          0.1710,  0.3849],
        [ 0.3688, -0.3691, -0.2495,  0.2255,  0.0332,  0.0599,  0.2841, -0.1064,
          0.0316,  0.1046],
        [-0.0225,  0.2275,  0.4450,  0.2718, -0.0148, -0.2924,  0.1063, -0.1068,
         -0.1863,  0.2524],
        [ 0.2863,  0.3567, -0.3763, -0.3134, -0.4016,  0.0257, -0.1893,  0.0494,
          0.2877, -0.0562]], grad_fn=<PermuteBackward0>)


In [60]:
w = hidden_layer.weight
b = hidden_layer.bias

f0 = torch.matmul(x, w.t()) + b
f0

tensor([[ 0.8685,  0.0075,  0.6002, -0.1228, -0.1678, -0.0743,  0.2652,  0.3329,
         -0.0544,  0.6975]], grad_fn=<AddBackward0>)

If we apply the hidden layer, we obtain the same results:

In [61]:
hidden_layer(x)

tensor([[ 0.8685,  0.0075,  0.6002, -0.1228, -0.1678, -0.0743,  0.2652,  0.3329,
         -0.0544,  0.6975]], grad_fn=<AddmmBackward0>)

![images/dnn_2.jpeg](images/dnn_2.jpeg) 

In order to introduce non-linearity into our net, allowing it to learn more complex patterns, we pass the output of the FC layer through the ReLU activation function, which looks like this:


![images/relu.png](images/relu.png) 

In [62]:
f0

tensor([[ 0.8685,  0.0075,  0.6002, -0.1228, -0.1678, -0.0743,  0.2652,  0.3329,
         -0.0544,  0.6975]], grad_fn=<AddBackward0>)

In [63]:
relu = nn.ReLU()

output = relu(f0)
output+

SyntaxError: invalid syntax (815518071.py, line 4)

In [ ]:
relu(hidden_layer(x))

tensor([[0.0000, 0.0000, 0.0851, 0.0000, 0.0000, 0.0000, 0.0000, 0.1146, 0.0000,
         0.3579]], grad_fn=<ReluBackward0>)

Now, we go through another round of linear multiplication from the nodes inside the hidden layer to the nodes in the output layer.

In [ ]:
output_layer = nn.Linear(10,3)

weights_2 = output_layer.weight 
bias_2 = output_layer.bias

output = torch.matmul(output, weights_2.t()) + bias_2
output

tensor([[-0.1700,  0.2526, -0.0515]], grad_fn=<AddBackward0>)

In [ ]:
output_layer(relu(hidden_layer(x)))

tensor([[-0.1700,  0.2526, -0.0515]], grad_fn=<AddmmBackward0>)

We have obtained a tensor of the same size of the output layer, which is the same size of the number of possible labels in our classification task.


In order to convert these values to probabilities, we apply the SoftMax activation function.

![images/softmax.png](images/softmax.png) 

In [ ]:
softmax = nn.Softmax(dim=1)

output = softmax(output)
output

tensor([[0.2738, 0.4179, 0.3083]], grad_fn=<SoftmaxBackward0>)

In [ ]:
softmax(output_layer(relu(hidden_layer(x))))

tensor([[0.2738, 0.4179, 0.3083]], grad_fn=<SoftmaxBackward0>)

The outputs are the probabilities of the sample being labeled with each one of the labels (0,1,2).

We manually implemented a forward pass of the network we defined at the start:

In [64]:
class ShallowNeuralNetwork(nn.Module):
    def __init__(self, input_size, num_hidden, output_size):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.fc1 = nn.Linear(input_size, num_hidden)
        self.output_layer = nn.Linear(num_hidden, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.output_layer(x)
        x = self.softmax(x)
        return x

In [65]:
shallow_nn = ShallowNeuralNetwork(5,10,3)

The values might be slightly different due to the randomization of the starting weights:

In [66]:
shallow_nn(x)

tensor([[0.3404, 0.3573, 0.3023]], grad_fn=<SoftmaxBackward0>)

In [67]:
shallow_nn.forward(x)

tensor([[0.3404, 0.3573, 0.3023]], grad_fn=<SoftmaxBackward0>)